This notebook contains slides for presenting some functionality and implementation details of the Logtalk Jupyter kernel.


It was created for a [RISE](https://rise.readthedocs.io/en/stable/index.html) slideshow started from Jupyter Notebook.

# A Jupyter Kernel for Prolog

- Execute queries

In [2]:
atom_length(atom,B).

B = 4

- Define predicates 

In [3]:
@user
my_append([], Res, Res).
my_append([H|T], List, [H|Res]) :-
  my_append(T, List, Res).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt loaded ]
% (0 warnings)

true

In [4]:
my_append([1,2], [3,4], R).

R = [1,2,3,4]

- Provides the possibility of executing Prolog code with Jupyter applications
- Replicates the standard Prolog usage and adds convenience functionality


- **Execute queries**: Example showing additional functionality: `atom_length(atom, L)`
    - Exact name? *Tab* &rarr; completion
    - Argument order? *Shift+Tab* &rarr; inspection 
    - Note: **missing terminating full-stop**
        - Eliminate a cause for queries not to be run right away


- Also: **Define predicates**


- Jupyter can be used to create notebooks consisting of cells like these ones
    - Source code and documentation
    - Create Assignments
    - Create slides for lectures like these ones
    - &rarr; Useful for teaching Prolog


- Before presenting implementation details: **General overview** of most important features for SWI-Prolog

## Differentiating Term Types

- A code cell can contain a query or multiple terms to be interpreted as directives and clauses to be added to a file. In the later case, the first line must be one of `@user`, `@user+`, `@file FILENAME`,  or `@file+ FILENAME`. The `+` variants append to an existing file instead of redefining it.

### Query
- Single term without body in a cell


In [5]:
X = [1,2,3], list::append(X, [4,5,6], Z).

X = [1,2,3],
Z = [1,2,3,4,5,6]

### Clause definition
- Any other term

In [6]:
@user
fact(a).
fact(b).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt already loaded; skipping ]
% (0 warnings)

true

In this case, previous clauses are replaced.

In [7]:
@user
fact(c).
fact(d).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt already loaded; skipping ]
% (0 warnings)

true

### Clause definition

- New clauses can be added instead by declaring the predicate `discontiguous`

In [1]:
@user
:- discontiguous(a/1).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt loaded ]
% (0 warnings)

true

In [2]:
@user+
a(1).
a(2).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt compiled ]
% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt reloaded ]
% (0 warnings)

true

In [3]:
@user+
b(3).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt compiled ]
% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt reloaded ]
% (0 warnings)

true

In [4]:
@user+
a(4).

% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt compiled ]
% [ /Users/pmoura/Documents/Logtalk/logtalk-jupyter-kernel/notebooks/slides/user.lgt reloaded ]
% (0 warnings)

true

In [5]:
listing(a/1), listing(b/1).

a(1).
a(2).
a(4).

b(3).


true

- A user might want to define a predicate in separate cells

## Handling Multiple Solutions

- Mimicking the usual backtracking mechanism

In [6]:
list::member(M, [a,b,c]).

M = a

In [7]:
jupyter::retry.

% Retrying goal: list::member(M,[a,b,c])


M = b

- Problem of the Jupyter kernel: **user intraction** not supported

## Debugging

Load an example, compiling it in debug mode:

In [3]:
set_logtalk_flag(debug, on), {elephants(loader)}.

% [ /Users/pmoura/logtalk/examples/elephants/elephants.lgt loaded ]
% [ /Users/pmoura/logtalk/examples/elephants/loader.lgt loaded ]
% (0 warnings)

yes

Try one of examples queries:

In [4]:
fred::number_of_legs(N).

N = 4

Repeat the query printing the trace of a goal, starting by recompiling the example in debug mode:

In [5]:
debugger::trace, fred::number_of_legs(N).

   Debugger switched on: tracing everything for all objects compiled in debug mode.
   Call: (1) fred::number_of_legs(_112771)
   Call: (2) number_of_legs(_112771)
   Fact: (2) number_of_legs(4)
   Exit: (2) number_of_legs(4)
   Exit: (1) fred::number_of_legs(4)

N = 4

Turn off tracing and compiling in debug mode:

In [6]:
debugger::notrace, set_logtalk_flag(debug, off).

   Debugger switched off.

yes

- Debugging cannot be performed interactively
- Instead, print the trace of a goal

## Running Tests

In [7]:
{ack(tester)}.

% 
% tests started at 2022-11-16, 21:43:53
% 
% running tests from object tests
% file: /Users/pmoura/logtalk/examples/ack/tests.lgt
% 
% ack_1: success (in 0.0020000000000000018 seconds)
% ack_2: success (in 0.006000000000000005 seconds)
% ack_3: success (in 0.028000000000000025 seconds)
% 
% 3 tests: 0 skipped, 3 passed, 0 failed (0 flaky)
% completed tests from object tests
% 
% 
% clause coverage ratio and covered clauses per entity predicate
% 
% ack: ack/3 - 3/3 - (all)
% ack: 3 out of 3 clauses covered, 100.000000% coverage
% 
% 1 entity declared as covered containing 3 clauses
% 1 out of 1 entity covered, 100.000000% entity coverage
% 3 out of 3 clauses covered, 100.000000% clause coverage
% 
% tests ended at 2022-11-16, 21:43:53
% 

yes

## Benchmarking Capabilities

- Whenever a query is executed, its runtime is stored in the database

In [13]:
list::member(M, [1,2,3]).

M = 1

In [14]:
jupyter::print_query_time.

Query:   list::member(M,[1,2,3])
Runtime: 0.0 s

true

- Access the previous goal and its runtime

## Structured Output

- Display all possible results of a goal in a table

In [15]:
jupyter::print_table((list::member(Member, [10,20,30,40]), Square is Member*Member)).

Member | Square | 
:- | :- | 
10 | 100 | 
20 | 400 | 
30 | 900 | 
40 | 1600 | 

true

## Introspection

- Predicate inspection: *Shift + Tab*
    - Help retrieved `help/1`

- Various `jupyter` predicates
    - Access documentation with a help predicate

In [16]:
jupyter::help.

jupyter::halt or halt

    Shuts down the running Prolog process.

    The next time code is to be executed, a new process is started.
    Everything defined in the database before does not exist anymore.

    Corresponds to the functionality of halt/0.
    Has the same effect as interrupting or restarting the Jupyter kernel.

--------------------------------------------------------------------------------

jupyter::help

    Outputs the documentation for all predicates from object jupyter.

--------------------------------------------------------------------------------

jupyter::print_query_time

    Prints the latest previous query and its runtime in seconds.

--------------------------------------------------------------------------------

jupyter::print_queries(+Ids)

    Prints previous queries which were executed in requests with IDs in Ids.

    Any $Var terms might be replaced by the variable's name.
    This is the case if a previous query with ID in Ids contains Var.
    Oth

true

- Various `juypter` (mostly convenience) predicates
- Difficult to remember all of them
    - In addition to completion and inspection: predicate to print all documentation

## Jupyter

- Originates from the **IPython** project
    - Enables interactive Python development
    - Several frontends, including a former version of **Jupyter Notebook**
        - Web application for handling Jupyter notebooks
        - Planned to be replaced by **JupyterLab**



- *Two-process model*:
<img style="float: right; max-width: 40%;" src="user_interaction_diagram.png">

    - Client process: responsible for user interaction
    - Kernel process: handles code execution

## Architecture

<img style="max-width: 80%;" src="architecture_diagram.png">






Kernel split in three:
- Extends IPython kernel: **inherits** the communication with a frontend via the ZeroMQ protocol


- Does not interpret Prolog itself
    - Starts an existing Prolog instance in a **subprocess**
        - Communicates with it according to the JSON-RPC 2.0 protocol
    - For any code execution **request**:
        - Sends a request message to the Prolog server containing the **code**
        - Prolog terms are read from the code and handled
    
    
- Make the kernel **extensible**: additional layer of a *kernel implementation* in between
    - **Responsible** for basically all functionality (e.g. handling Prolog **server**)
    - For every request the kernel receives, a **method** of the implementation class is called
    - Kernel started: loads **config** file
        - Can contain paths to interpreter-specific Python class files
    - By **extending** default implementation class and **overriding** methods
        - Kernel behaviour can be adjusted
    - Had to be done to support predicate inspection for both    
- Configure to start a different Prolog server

## Changing the Prolog Implementation

- Switch between Prolog backends on the fly
- The previous server process is kept running
    - When switching back, the database state has not changed

In [2]:
sicstus.

true

In [ ]:
app([1,2], [3], R).

   Debugger is off.

!     Unknown error message for component jupyter: error(existence_error(procedure,user:app/3),existence_error($@(user:app([1,2],[3],_47137),4436819036),0,procedure,user:app/3,0))


In [1]:
swi.

true

In [ ]:
app([1,2], [3], R)

## Extending the Kernel

- At first, the kernel was developed for SICStus Prolog only
    - Adjusted for SWI-Prolog as well
    - Made extensible for further Prolog backends
    

- By **replacing the Prolog server**, the Python part can easily support a different implementation
    - Requirements: receive requests as JSON-RPC 2.0 messages, handle them, and send responses
    - Might be possible to further extend the existing server with conditional compilation
        - Advanced features might require significant changes

- By **overriding the `LogtalkKernelBaseImplementation` class**, most of the basic kernel behaviour can be adjusted
    - For SICStus and SWI-Prolog, the handling of predicate inspection differs

- Server replacement:
    - Most code compatible with SICStus and SWI &rarr; **conditional compilation**
    - Expected to be similar for other implementations
    - Extend existing:
        - Implementing **basic code execution** should not require major effort
        - More **advanced features** might involve significant changes


- Replacement of the server does not suffice?: Python extension
- In case of SWI- and SICStus, the only Python code that differs is for predicate inspection

## Future Work

- Support further Prolog backends
    - Or multiple versions of the same implementation

- Combine strengths of several Prolog instances
    - Kernel can be connected with multiple servers at once
    - Reusing results for another one should be relatively easy

- Send commands to all available Prolog servers *at once*
    - Detect differences in the behaviour
    - Compare the performance by using the benchmarking functionality